# Once your Groundtruth job has completed, lets prepare for training

This notebook walks you through the steps we have taken to process the object detection label output from Ground Truth to prepare it for model training in SageMaker. 

1. [Join together outputs from multiple labeling jobs](#join_output)
1. [Filter out labels that did not meet our quality bar](#filter_bad_labels)
1. [Inject class labels (if you didn't have the Ground Truth workers pick classes)](#inject_class)
1. [Split train/validation data](#split_train)
1. [data augmentation](#data_aug)

## Setup

In [1]:
BUCKET = 'cvml-sagemaker-repo'
JOB_NAME = 'wakeboarder-detection' 

### Import dependencies and define helper functions

In [2]:
import numpy as np
import random
import os, shutil
import json
import boto3
import botocore
import sagemaker

In [3]:
sagemaker_client = boto3.client('sagemaker')

def make_tmp_folder(folder_name):
    try:
        os.makedirs(folder_name, exist_ok=False)
    except FileExistsError:
        print("{} folder already exists".format(folder_name))
        
def read_manifest_file(file_path):
    with open(file_path, 'r') as f:
        output = [json.loads(line.strip()) for line in f.readlines()]
        return output

### Specify the Ground Truth labeling job id(s) 

In [4]:
## if using your own Ground Truth labeling job, replace below with appropriate job IDs
LABEL_JOB_IDS = ['wakeboarder-detection']


In [5]:
TMP_FOLDER_NAME = 'tmp'
make_tmp_folder(TMP_FOLDER_NAME)


## 1. Enable Flexibility and Join outputs from multiple jobs <a id='join_output'></a>

To be able to iterate on Ground Truth jobs, this process can join outputs from multiple jobs. You can still run this cell if you have only 1 job. 

The below code takes one or more Ground Truth job IDs, download the output (Augmented Manifest File format) and join them together into one array for manipulation 

In [7]:
joined_outputs = []

def get_output_manifest_s3_uri(label_job_id):
    # uncomment below if you are using your own Ground Truth labeling job 
     return sagemaker_client.describe_labeling_job(LabelingJobName=label_job_id)['LabelingJobOutput']['OutputDatasetS3Uri']

for label_job_id in LABEL_JOB_IDS: 
    output_manifest_s3_uri = get_output_manifest_s3_uri(label_job_id)
    output_manifest_fname = "{}-{}".format(label_job_id, os.path.split(output_manifest_s3_uri)[1])
    !aws s3 cp $output_manifest_s3_uri $TMP_FOLDER_NAME/$output_manifest_fname
    output_manifest_local_path = os.path.join(TMP_FOLDER_NAME, output_manifest_fname)
    output_manifest_lines = read_manifest_file(output_manifest_local_path)
    print("loaded {} lines from {}".format(len(output_manifest_lines), output_manifest_local_path))
    joined_outputs += output_manifest_lines
    
print("loaded total of {} lines".format(len(joined_outputs)))

download: s3://cvml-sagemaker-repo/ground_truth/output/wakeboarder-detection/manifests/output/output.manifest to tmp/wakeboarder-detection-output.manifest
loaded 298 lines from tmp/wakeboarder-detection-output.manifest
loaded total of 298 lines


## View Example labels

In [8]:
joined_outputs[15]

{'source-ref': 's3://cvml-sagemaker-repo/frames/wakeboarding_001055.jpg',
 'wakeboarder-detection': {'annotations': [{'class_id': 0,
    'width': 483,
    'top': 125,
    'height': 544,
    'left': 306}],
  'image_size': [{'width': 1280, 'depth': 3, 'height': 720}]},
 'wakeboarder-detection-metadata': {'job-name': 'labeling-job/wakeboarder-detection',
  'class-map': {'0': 'wakeboard'},
  'human-annotated': 'yes',
  'objects': [{'confidence': 0.09}],
  'creation-date': '2020-03-12T20:02:31.826196',
  'type': 'groundtruth/object-detection'}}

In [9]:
joined_outputs[-15]

{'source-ref': 's3://cvml-sagemaker-repo/frames/wakeboarding_018743.jpg',
 'wakeboarder-detection': {'annotations': [],
  'image_size': [{'width': 1280, 'depth': 3, 'height': 720}]},
 'wakeboarder-detection-metadata': {'job-name': 'labeling-job/wakeboarder-detection',
  'class-map': {},
  'human-annotated': 'yes',
  'objects': [],
  'creation-date': '2020-03-12T20:45:58.452830',
  'type': 'groundtruth/object-detection'}}

## 2. MANUAL REVIEW: Discard any bad labels from visual inspection <a id="filter_bad_labels"></a>

In the __Sagemaker console__, under labeling jobs, select your completed labeling job name. Review any images that don't meet the quality bar and you can remove them from the list by placing them in the list array below: (remove any .jpg file extensions)

In [10]:
TO_DISCARD = set([
    'wakeboarding_003299',
    'wakeboarding_007127',
    'wakeboarding_007391',
    'wakeboarding_008183',
    'wakeboarding_008777',
    'wakeboarding_013331',
    'wakeboarding_013397',
    'wakeboarding_013925',
    'wakeboarding_015905',
    'wakeboarding_015971',
    'wakeboarding_017555',
    'wakeboarding_017753',
    'wakeboarding_017819',
    'wakeboarding_018149',
    'wakeboarding_018611',
    'wakeboarding_018677',
    'wakeboarding_018743',
    'wakeboarding_019535'
    
])

In [11]:
filtered_manifest = []
count_filtered = 0
for line in joined_outputs:
    filename= os.path.split(line["source-ref"])[1]
    imageid = os.path.splitext(filename)[0]
    if imageid not in TO_DISCARD:
        filtered_manifest.append(line)
    else:
        count_filtered+=1
        
print("filtered out {} labels. {} labels remains".format(count_filtered, len(filtered_manifest)))

filtered out 18 labels. 280 labels remains


In [12]:
## example entry
filtered_manifest[2]

{'source-ref': 's3://cvml-sagemaker-repo/frames/wakeboarding_000197.jpg',
 'wakeboarder-detection': {'annotations': [{'class_id': 0,
    'width': 256,
    'top': 75,
    'height': 469,
    'left': 538}],
  'image_size': [{'width': 1280, 'depth': 3, 'height': 720}]},
 'wakeboarder-detection-metadata': {'job-name': 'labeling-job/wakeboarder-detection',
  'class-map': {'0': 'wakeboard'},
  'human-annotated': 'yes',
  'objects': [{'confidence': 0.09}],
  'creation-date': '2020-03-12T21:09:47.779834',
  'type': 'groundtruth/object-detection'}}

## 3. Split dataset between train and validation <a id='split_train'></a>

SageMaker requires two datasets during training: train and validation dataset. The training set consists of the images and annotations you want to actually train the model with. The validation set is not used for training but used to “validate” that each training pass is improving the accuracy of the model and compare accuracy between different training jobs during hyper-parameter tuning. 

In [13]:
def train_validation_split(labels, split_factor=0.9):
    np.random.shuffle(labels)

    dataset_size = len(labels)
    train_test_split_index = round(dataset_size*split_factor)

    train_data = labels[:train_test_split_index]
    validation_data = labels[train_test_split_index:]
    return train_data, validation_data

In [15]:
train_data, validation_data = train_validation_split(np.array(filtered_manifest), split_factor=0.9)

print("training data size:{}\nvalidation data size:{}".format(train_data.shape[0], validation_data.shape[0]))

training data size:252
validation data size:28


In [16]:
with open(os.path.join(TMP_FOLDER_NAME, 'train.manifest'), 'w') as f:
    for line in train_data:
        f.write(json.dumps(line))
        f.write('\n')
    
with open(os.path.join(TMP_FOLDER_NAME,'validation.manifest'), 'w') as f:
    for line in validation_data:
        f.write(json.dumps(line))
        f.write('\n')

In [17]:
!wc -l $TMP_FOLDER_NAME/train.manifest
!wc -l $TMP_FOLDER_NAME/validation.manifest

252 tmp/train.manifest
28 tmp/validation.manifest


In [22]:
!aws s3 cp $TMP_FOLDER_NAME/train.manifest s3://$BUCKET/training-manifest/$JOB_NAME/train.manifest
!aws s3 cp $TMP_FOLDER_NAME/validation.manifest s3://$BUCKET/training-manifest/$JOB_NAME/validation.manifest

upload: tmp/train.manifest to s3://cvml-sagemaker-repo/training-manifest/wakeboarder-detection/train.manifest
upload: tmp/validation.manifest to s3://cvml-sagemaker-repo/training-manifest/wakeboarder-detection/validation.manifest


## 5. OPTIONAL: Data augmentation <a id='data_aug'></a>

In [19]:
%%time
%run ./flip_images.py -m s3://$BUCKET/training-manifest/$JOB_NAME/train.manifest -d $TMP_FOLDER_NAME -b $BUCKET

INFO:__main__:working directory: tmp
INFO:__main__:image: wakeboarding_003497
INFO:__main__:augmentation does not exist: s3://cvml-sagemaker-repo/frames/ccw/wakeboarding_003497-ccw-rotate.jpg


252


INFO:__main__:wrote image to tmp/wakeboarding_003497-x-flip.jpg
INFO:common_utils:wrote to s3://cvml-sagemaker-repo/frames/x-flipped/wakeboarding_003497-x-flip.jpg
INFO:__main__:wrote image to tmp/wakeboarding_003497-y-flip.jpg
INFO:common_utils:wrote to s3://cvml-sagemaker-repo/frames/y-flipped/wakeboarding_003497-y-flip.jpg
INFO:__main__:wrote image to tmp/wakeboarding_003497-cw-rotate.jpg
INFO:common_utils:wrote to s3://cvml-sagemaker-repo/frames/cw/wakeboarding_003497-cw-rotate.jpg
INFO:__main__:wrote image to tmp/wakeboarding_003497-ccw-rotate.jpg
INFO:common_utils:wrote to s3://cvml-sagemaker-repo/frames/ccw/wakeboarding_003497-ccw-rotate.jpg
INFO:__main__:image: wakeboarding_014123
INFO:__main__:augmentation does not exist: s3://cvml-sagemaker-repo/frames/ccw/wakeboarding_014123-ccw-rotate.jpg
INFO:__main__:wrote image to tmp/wakeboarding_014123-x-flip.jpg
INFO:common_utils:wrote to s3://cvml-sagemaker-repo/frames/x-flipped/wakeboarding_014123-x-flip.jpg
INFO:__main__:wrote imag

INFO:__main__:wrote image to tmp/wakeboarding_001979-y-flip.jpg
INFO:common_utils:wrote to s3://cvml-sagemaker-repo/frames/y-flipped/wakeboarding_001979-y-flip.jpg
INFO:__main__:wrote image to tmp/wakeboarding_001979-cw-rotate.jpg
INFO:common_utils:wrote to s3://cvml-sagemaker-repo/frames/cw/wakeboarding_001979-cw-rotate.jpg
INFO:__main__:wrote image to tmp/wakeboarding_001979-ccw-rotate.jpg
INFO:common_utils:wrote to s3://cvml-sagemaker-repo/frames/ccw/wakeboarding_001979-ccw-rotate.jpg
INFO:__main__:image: wakeboarding_010559
INFO:__main__:augmentation does not exist: s3://cvml-sagemaker-repo/frames/ccw/wakeboarding_010559-ccw-rotate.jpg
INFO:__main__:wrote image to tmp/wakeboarding_010559-x-flip.jpg
INFO:common_utils:wrote to s3://cvml-sagemaker-repo/frames/x-flipped/wakeboarding_010559-x-flip.jpg
INFO:__main__:wrote image to tmp/wakeboarding_010559-y-flip.jpg
INFO:common_utils:wrote to s3://cvml-sagemaker-repo/frames/y-flipped/wakeboarding_010559-y-flip.jpg
INFO:__main__:wrote imag

INFO:__main__:wrote image to tmp/wakeboarding_011351-cw-rotate.jpg
INFO:common_utils:wrote to s3://cvml-sagemaker-repo/frames/cw/wakeboarding_011351-cw-rotate.jpg
INFO:__main__:wrote image to tmp/wakeboarding_011351-ccw-rotate.jpg
INFO:common_utils:wrote to s3://cvml-sagemaker-repo/frames/ccw/wakeboarding_011351-ccw-rotate.jpg
INFO:__main__:image: wakeboarding_001187
INFO:__main__:augmentation does not exist: s3://cvml-sagemaker-repo/frames/ccw/wakeboarding_001187-ccw-rotate.jpg
INFO:__main__:wrote image to tmp/wakeboarding_001187-x-flip.jpg
INFO:common_utils:wrote to s3://cvml-sagemaker-repo/frames/x-flipped/wakeboarding_001187-x-flip.jpg
INFO:__main__:wrote image to tmp/wakeboarding_001187-y-flip.jpg
INFO:common_utils:wrote to s3://cvml-sagemaker-repo/frames/y-flipped/wakeboarding_001187-y-flip.jpg
INFO:__main__:wrote image to tmp/wakeboarding_001187-cw-rotate.jpg
INFO:common_utils:wrote to s3://cvml-sagemaker-repo/frames/cw/wakeboarding_001187-cw-rotate.jpg
INFO:__main__:wrote image

...

INFO:__main__:wrote image to tmp/wakeboarding_017885-y-flip.jpg
INFO:common_utils:wrote to s3://cvml-sagemaker-repo/frames/y-flipped/wakeboarding_017885-y-flip.jpg
INFO:__main__:wrote image to tmp/wakeboarding_017885-cw-rotate.jpg
INFO:common_utils:wrote to s3://cvml-sagemaker-repo/frames/cw/wakeboarding_017885-cw-rotate.jpg
INFO:__main__:wrote image to tmp/wakeboarding_017885-ccw-rotate.jpg
INFO:common_utils:wrote to s3://cvml-sagemaker-repo/frames/ccw/wakeboarding_017885-ccw-rotate.jpg
INFO:__main__:image: wakeboarding_015179
INFO:__main__:augmentation does not exist: s3://cvml-sagemaker-repo/frames/ccw/wakeboarding_015179-ccw-rotate.jpg
INFO:__main__:wrote image to tmp/wakeboarding_015179-x-flip.jpg
INFO:common_utils:wrote to s3://cvml-sagemaker-repo/frames/x-flipped/wakeboarding_015179-x-flip.jpg
INFO:__main__:wrote image to tmp/wakeboarding_015179-y-flip.jpg
INFO:common_utils:wrote to s3://cvml-sagemaker-repo/frames/y-flipped/wakeboarding_015179-y-flip.jpg
INFO:__main__:wrote imag

INFO:__main__:wrote image to tmp/wakeboarding_018083-cw-rotate.jpg
INFO:common_utils:wrote to s3://cvml-sagemaker-repo/frames/cw/wakeboarding_018083-cw-rotate.jpg
INFO:__main__:wrote image to tmp/wakeboarding_018083-ccw-rotate.jpg
INFO:common_utils:wrote to s3://cvml-sagemaker-repo/frames/ccw/wakeboarding_018083-ccw-rotate.jpg
INFO:__main__:image: wakeboarding_015773
INFO:__main__:augmentation does not exist: s3://cvml-sagemaker-repo/frames/ccw/wakeboarding_015773-ccw-rotate.jpg
INFO:__main__:wrote image to tmp/wakeboarding_015773-x-flip.jpg
INFO:common_utils:wrote to s3://cvml-sagemaker-repo/frames/x-flipped/wakeboarding_015773-x-flip.jpg
INFO:__main__:wrote image to tmp/wakeboarding_015773-y-flip.jpg
INFO:common_utils:wrote to s3://cvml-sagemaker-repo/frames/y-flipped/wakeboarding_015773-y-flip.jpg
INFO:__main__:wrote image to tmp/wakeboarding_015773-cw-rotate.jpg
INFO:common_utils:wrote to s3://cvml-sagemaker-repo/frames/cw/wakeboarding_015773-cw-rotate.jpg
INFO:__main__:wrote image

INFO:__main__:wrote image to tmp/wakeboarding_010625-ccw-rotate.jpg
INFO:common_utils:wrote to s3://cvml-sagemaker-repo/frames/ccw/wakeboarding_010625-ccw-rotate.jpg
INFO:__main__:image: wakeboarding_000065
INFO:__main__:augmentation does not exist: s3://cvml-sagemaker-repo/frames/ccw/wakeboarding_000065-ccw-rotate.jpg
INFO:__main__:wrote image to tmp/wakeboarding_000065-x-flip.jpg
INFO:common_utils:wrote to s3://cvml-sagemaker-repo/frames/x-flipped/wakeboarding_000065-x-flip.jpg
INFO:__main__:wrote image to tmp/wakeboarding_000065-y-flip.jpg
INFO:common_utils:wrote to s3://cvml-sagemaker-repo/frames/y-flipped/wakeboarding_000065-y-flip.jpg
INFO:__main__:wrote image to tmp/wakeboarding_000065-cw-rotate.jpg
INFO:common_utils:wrote to s3://cvml-sagemaker-repo/frames/cw/wakeboarding_000065-cw-rotate.jpg
INFO:__main__:wrote image to tmp/wakeboarding_000065-ccw-rotate.jpg
INFO:common_utils:wrote to s3://cvml-sagemaker-repo/frames/ccw/wakeboarding_000065-ccw-rotate.jpg
INFO:__main__:image: w

INFO:__main__:image: wakeboarding_011615
INFO:__main__:augmentation does not exist: s3://cvml-sagemaker-repo/frames/ccw/wakeboarding_011615-ccw-rotate.jpg
INFO:__main__:wrote image to tmp/wakeboarding_011615-x-flip.jpg
INFO:common_utils:wrote to s3://cvml-sagemaker-repo/frames/x-flipped/wakeboarding_011615-x-flip.jpg
INFO:__main__:wrote image to tmp/wakeboarding_011615-y-flip.jpg
INFO:common_utils:wrote to s3://cvml-sagemaker-repo/frames/y-flipped/wakeboarding_011615-y-flip.jpg
INFO:__main__:wrote image to tmp/wakeboarding_011615-cw-rotate.jpg
INFO:common_utils:wrote to s3://cvml-sagemaker-repo/frames/cw/wakeboarding_011615-cw-rotate.jpg
INFO:__main__:wrote image to tmp/wakeboarding_011615-ccw-rotate.jpg
INFO:common_utils:wrote to s3://cvml-sagemaker-repo/frames/ccw/wakeboarding_011615-ccw-rotate.jpg
INFO:__main__:image: wakeboarding_002045
INFO:__main__:augmentation does not exist: s3://cvml-sagemaker-repo/frames/ccw/wakeboarding_002045-ccw-rotate.jpg
INFO:__main__:wrote image to tmp/

CPU times: user 3min 6s, sys: 12.6 s, total: 3min 19s
Wall time: 2min 32s


In [21]:
%run ./flip_annotations.py -m s3://$BUCKET/training-manifest/$JOB_NAME/train.manifest -d $TMP_FOLDER_NAME -p $JOB_NAME

INFO:__main__:working directory: tmp


252


KeyError: 'bb'

# Next step

Now we are ready to start training jobs! Move on to the [next notebook](./03_Sagemaker_Training_API.ipynb) to submit a sagemaker training job to train our custom object detection model!